In [2]:
%load_ext dotenv
%dotenv

## Chaining/Piping

In [4]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [71]:
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    seed=100
)

In [14]:
instructions_for_parsing = CommaSeparatedListOutputParser().get_format_instructions()

In [16]:
chat_template = ChatPromptTemplate.from_messages([('human',"I have just bought a pet {pet}, could you please suggest 3 names? \n" + instructions_for_parsing)])

In [18]:
chat_template

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template='I have just bought a pet {pet}, could you please suggest 3 names? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'), additional_kwargs={})])

In [20]:
output_comma_parser = CommaSeparatedListOutputParser()

In [22]:
chain = chat_template | llm | output_comma_parser

In [24]:
response = chain.invoke({'pet':'snake'})

In [28]:
response

['Slytherin', 'Serpentina', 'Hiss-teria']

## Batching

In [81]:
chat_template_b = ChatPromptTemplate.from_messages([('human',"I have recently adopted a {pet} of the breed {breed}, could you suggest how I should go about training?")])

In [79]:
chain_b = chat_template_b | llm

In [38]:
%%time
chain_b.invoke({'pet':'dog', 'breed':'doberman'})

CPU times: user 19.7 ms, sys: 2.96 ms, total: 22.6 ms
Wall time: 2.16 s


AIMessage(content='Congratulations on adopting a Doberman! They are intelligent, loyal, and energetic dogs that can make wonderful companions. Here are some tips for training your Doberman:\n\n### 1. **Start Early**\n   - Begin training as soon as you bring your Doberman home. Early socialization and training are crucial for developing a well-adjusted dog.\n\n### 2. **Socialization**\n   - Expose your Doberman to various environments, people, and other animals. This helps them become well-rounded and reduces the likelihood of fear or aggression.\n\n### 3. **Positive Reinforcement**\n   - Use positive reinforcement techniques, such as treats, praise, and play, to reward good behavior. This encourages your dog to repeat those behaviors.\n\n### ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 30, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 

In [44]:
%%time
chain_b.invoke({'pet':'snake', 'breed':'ball python'})

CPU times: user 16.7 ms, sys: 2.68 ms, total: 19.3 ms
Wall time: 2.99 s


AIMessage(content='Training a ball python is quite different from training a dog or other pets, as snakes do not respond to commands or training in the same way. However, you can help your ball python become more comfortable with handling and its environment. Here are some tips for "training" your ball python:\n\n### 1. **Handling:**\n   - **Start Slowly:** Allow your snake to acclimate to its new environment for at least a week before handling. This helps reduce stress.\n   - **Gentle Handling:** When you start handling, do so gently and support the snake\'s body. Avoid sudden movements that could startle it.\n   - **Short Sessions:** Keep handling sessions short (10-15 minutes) at first, gradually increasing the time as', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 29, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_

In [45]:
%%time
chain_b.batch([{'pet':'dog', 'breed':'doberman'},{'pet':'snake', 'breed':'ball python'}])

CPU times: user 28.3 ms, sys: 4.44 ms, total: 32.7 ms
Wall time: 2.35 s


[AIMessage(content='Congratulations on adopting a Doberman! They are intelligent, loyal, and energetic dogs that can make wonderful companions. Here are some tips for training your Doberman:\n\n### 1. **Start Early**\n   - Begin training as soon as you bring your Doberman home. Early socialization and training are crucial for developing a well-adjusted dog.\n\n### 2. **Socialization**\n   - Expose your Doberman to various environments, people, and other animals. This helps them become well-rounded and reduces the likelihood of fear or aggression.\n\n### 3. **Positive Reinforcement**\n   - Use positive reinforcement techniques, such as treats, praise, and play, to reward good behavior. This encourages your dog to repeat those behaviors.\n\n### ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 30, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0,

## Streaming

In [83]:
response = chain_b.stream({'pet':'dog', 'breed':'doberman'})

In [85]:
response

<generator object RunnableSequence.stream at 0x115651c60>

In [87]:
for i in response:
    print(i.content, end='')

Congratulations on adopting a Doberman! They are intelligent, loyal, and energetic dogs that can make wonderful companions. Here are some tips for training your Doberman:

### 1. **Start Early**
   - Begin training as soon as you bring your dog home. Early socialization and training are crucial for Dobermans, as they are sensitive and can develop behavioral issues if not properly trained.

### 2. **Use Positive Reinforcement**
   - Reward-based training is the most effective method. Use treats, praise, and playtime as rewards for good behavior. This encourages your dog to repeat those behaviors.

### 3. **Be Consistent**
   - Use the same commands and cues consistently. This helps your dog understand what you expect from them. Consistency in your training approach is key.

### 4. **Socialization**
   - Expose your Doberman to different environments, people, and other animals. This helps them become well-adjusted and reduces the likelihood of fear-based behaviors.

### 5. **Basic Comman